In [4]:
#python packages

#Bert Model
from transformers import pipeline, AutoTokenizer, BertForMaskedLM
unmasker = pipeline('fill-mask', model='dumitrescustefan/bert-base-romanian-cased-v1', top_k=10)

#tokenizer
import spacy
NLP = spacy.load("ro_core_news_lg")

#text classification
import fasttext.util
fasttext.util.download_model('ro', if_exists='ignore')
ft = fasttext.load_model('cc.ro.300.bin')

#zipf_score
from wordfreq import zipf_frequency

#cosine similarity
import scipy.spatial, scipy.special

#min max scaler
from sklearn.preprocessing import MinMaxScaler

C:\Users\Anghel Fabian\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [220]:
import json

synonyms_dict = {}
antonyms_dict = {}

#load synonyms and antonyms
#with open(join(dirname(__file__), 'synonyms.json'), 'r') as f:
with open('scraping/synonyms.json', 'r') as f:
    synonyms_dict = json.load(f)

#with open(join(dirname(__file__), 'antonyms.json'), 'r') as f:
with open('scraping/antonyms.json', 'r') as f:
    antonyms_dict = json.load(f)

In [221]:
#load dex online data
with open('dex-online-database/word_inflections.json', 'r') as f:
    word_inflections = json.load(f)

with open('dex-online-database/spaCy_tags.json', 'r') as f:
    spacy_tags = json.load(f)

In [222]:
def preprocess_sentence(sentence):
    #get right versions of ș/ț
    sentence = sentence.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș")

    #transform a/i + unicode 770/u+0302 in â/î
    sentence = sentence.replace("a\u0302", "â").replace("i\u0302", "î").replace("A\u0302", "Â").replace("I\u0302", "Î")

    #transform s/t + unicode 807/u+0327 or 806/u+0326 in ș/ț
    sentence = sentence.replace("s\u0327", "ș").replace("t\u0327", "ț").replace("S\u0327", "Ș").replace("T\u0327", "Ț")
    sentence = sentence.replace("s\u0326", "ș").replace("t\u0326", "ț").replace("S\u0326", "Ș").replace("T\u0326", "Ț")

    #transform a + unicode 774/u+0306 in ă
    sentence = sentence.replace("a\u0306", "ă").replace("A\u0307", "Ă")

    return sentence


In [223]:
def valid_token(token, zipf_score):
    #if token is punctuation
    if token.is_punct:
        return False
    
    #if token is number
    if token.is_digit or token.like_num:
        return False
    
    #if token is whitespace
    if token.is_space:
        return False
    
    #if token is currency:
    if token.is_currency:
        return False
    
    #if token is url or email
    if token.like_url or token.like_email:
        return False
    
    #should be replaced only tokens that are nouns, adjectives, adverbs or verbs (proper nouns are also excluded here)
    if token.pos_ not in ["NOUN", "ADJ", "ADV", "VERB", "AUX"]:
        return False
    
    #if token is proper noun
    if token.pos_ == "NOUN" and token.tag_[1] == 'p':
        return False
    
    #too high zipf score
    if zipf_score > 4.25:
    #if zipf_score > 4.5:
    #if zipf_score > 4:
        return False
    
    #to do: check ENTs
    #some can be general words (like "îmbrăcăminte", "litoral", "alcool")
    #DATETIME should still be replaced
    
    return True

In [224]:
def word_candidates(sentence, new_tokens):
    tokens = NLP(sentence)
    replacable_tokens = []

    #sort tokens by Zipf score
    for token in tokens:
        #token_score = zipf_frequency(token.text, 'ro')
        token_score = zipf_frequency(token.lemma_, "ro")
        if valid_token(token, token_score) and token.text not in new_tokens:
            #context = extract_context(tokens, token.i)
            replacable_tokens.append((token, token_score))

    replacable_tokens.sort(key=lambda x: x[1])

    return replacable_tokens

In [225]:
def substitution_generation(sentence, token):
    # generate a masked sentence in this form: [CLS] original sentence [SEP] masked sentence [SEP]
    masked_sentence = sentence[:token.idx] + "[MASK]" + sentence[token.idx + len(token.text):]
    model_sentence = "[CLS] " + sentence + " [SEP] " + masked_sentence + " [SEP]"
    result = unmasker(model_sentence)
    suggestions = []

    for x in result:
        #check if suggestion is same part of speech 
        replaced_sentence = sentence[:token.idx] + x["token_str"] + sentence[token.idx + len(token.text):]
        replaced_tokens = NLP(replaced_sentence)
        suggestion_token = replaced_tokens[token.i]

        if token.pos != suggestion_token.pos and not (token.pos_ == "AUX" and suggestion_token.pos_ == "VERB") and not (token.pos_ == "VERB" and suggestion_token.pos_ == "AUX"):
            continue

        suggestions.append(x)

    return suggestions    

In [226]:
def synonym_score(suggestion, token):
    sugg_token = NLP(suggestion)[0]
    sugg_lemma = sugg_token.lemma_
    token_lemma = token.lemma_

    #positive score if synonyms
    if (token_lemma in synonyms_dict and sugg_lemma in synonyms_dict[token_lemma]) or (sugg_lemma in synonyms_dict and token_lemma in synonyms_dict[sugg_lemma]):
        return 1

    #negative score if antonyms
    if (token_lemma in antonyms_dict and sugg_lemma in antonyms_dict[token_lemma]) or (sugg_lemma in antonyms_dict and token_lemma in antonyms_dict[sugg_lemma]):
        return 0
    if "ne" + token_lemma == sugg_lemma or "ne" + sugg_lemma == token_lemma:
        return 0
    if "in" + token_lemma == sugg_lemma or "in" + sugg_lemma == token_lemma:
        return 0
    
    #negative score if same word to encourage different suggestions
    if token_lemma == sugg_lemma:
        return 0.25
    
    #netural score if not related
    return 0.5

In [227]:
def substitution_ranking(suggetions, token):
    substitutions = []

    bert_scores = []
    zipf_scores = []
    fasttext_scores = []
    synonym_scores = []

    # calculate a score for each suggestion based on BERT, Zipf and FastText
    for x in suggetions:
        bert_scores.append([x["score"]])

        zipf_score = zipf_frequency(x["token_str"], 'ro')
        zipf_scores.append([zipf_score])

        fasttext_score = 1 - scipy.spatial.distance.cosine(ft[token.text], ft[x["token_str"]])
        fasttext_scores.append([fasttext_score])

        syn_score = synonym_score(x["token_str"], token)
        synonym_scores.append([syn_score])
    
    if len(bert_scores) > 0:
        bert_scaler = MinMaxScaler()
        zipf_scaler = MinMaxScaler()
        fasttext_scaler = MinMaxScaler()
        synonym_scaler = MinMaxScaler()

        bert_scores_norm = bert_scaler.fit_transform(bert_scores)
        zipf_scores_norm = zipf_scaler.fit_transform(zipf_scores)
        fasttext_scores_norm = fasttext_scaler.fit_transform(fasttext_scores)
        synonym_scores_norm = synonym_scaler.fit_transform(synonym_scores)

        for i in range(len(suggetions)):
            final_score = (bert_scores_norm[i] + zipf_scores_norm[i] + fasttext_scores_norm[i] + synonym_scores[i]) / 4
            substitutions.append((suggetions[i]["token_str"], final_score))

        # sort suggestions by score and replace the word with the best suggestion if it has a higher Zipf score
        substitutions.sort(reverse=True, key=lambda x: x[1])
        #print(substitutions)

    return substitutions

In [246]:
def align_word_form(token, suggestion):
    #search if original word has inflections available for its' pos tag
    has_inflections = False
    inflection_ids = []

    if token.lemma_ in word_inflections:
        if token.tag_ in spacy_tags:
            has_inflections = True
            inflection_ids = spacy_tags[token.tag_]["dex_online_id"]

    #replace the top replacement with the right form if found
    suggestion_lemma = NLP(suggestion)[0].lemma_

    if has_inflections and suggestion_lemma in word_inflections:
        for id in inflection_ids:
            str_id = str(id)
            if str_id in word_inflections[suggestion_lemma]["inflections"]:
                if token.is_title:
                    return word_inflections[suggestion_lemma]["inflections"][str_id].title()
                return word_inflections[suggestion_lemma]["inflections"][str_id]

    #if not replacement is found return original word
    if token.is_title:
        return suggestion.title()
    return suggestion

In [257]:
def sentence_simplification(sentence):
    token_replaced = True
    new_tokens = set([])

    #preprocessing
    sentence = preprocess_sentence(sentence)

    #sent_tokens = NLP(sentence)
    #for token in sent_tokens:
        #if token.ent_iob != 2:
            #print(token.text, token.ent_type_)
        #print(token.text, token.pos_, token.tag_)

    while token_replaced:
        token_replaced = False

        replacable_tokens = word_candidates(sentence, new_tokens)
        
        for token in replacable_tokens:
            if token_replaced:
                break

            token_text = token[0].text

            suggestions = substitution_generation(sentence, token[0])
            suggestions = substitution_ranking(suggestions, token[0])

            if len(suggestions) == 0:
                continue
            
            top_replacement = suggestions[0][0]

            if zipf_frequency(top_replacement, 'ro') >= zipf_frequency(token_text, 'ro'):
                top_replacement = align_word_form(token[0], top_replacement)
                sentence = sentence[:token[0].idx] + top_replacement + sentence[token[0].idx + len(token_text):]
                new_tokens.add(top_replacement)
                token_replaced = True

    return sentence

In [258]:
def text_simplification(text):
    doc = NLP(text)
    for sentence in doc.sents:
        new_sentence = sentence_simplification(sentence.text)
        text = text.replace(sentence.text, new_sentence)
    return text

In [259]:
#normal sentences
sentence_list = ["Priveliștea era spectaculoasă și se întindea până la orizont.", 
                 "Ei consumaseră prea mult alcool în acea seară.", 
                 "Mihai a mers la mall ca să își achiziționeze îmbrăcăminte.",
                 "Eu și gașca mea am mers în vacanță la litoral.",
                 "Am mâncat prea multă ciocolată și am făcut hiperglicemie.", 
                 "La facultate am studiat despre algoritmi și structuri de date.",
                 "Articolul publicat de mine a fost citit de toată lumea.",]

for sentence in sentence_list:
    print(sentence_simplification(sentence))

Marea era spectaculoasă și se întindea până la orizont.
Ei avuseră prea mult alcool în acea seară.
Mihai a mers la mall ca să își cumpere haină.
Eu și gașca mea am mers în vacanță la mare.
Am mâncat prea multă ciocolată și am făcut diabet.
La facultate am învățat despre structuri și structuri de date.
Articolul publicat de mine a fost citit de toată lumea.


In [260]:
#complex sentences
sentence_list = ["Procrastinarea constantă în fața responsabilităților academice poate duce la o exacerbare a anxietății și la compromiterea performanțelor intelectuale pe termen lung.",
                 "Implementarea unor strategii pedagogice inovatoare poate facilita internalizarea cunoștințelor teoretice într-un mod eficient și sustenabil.",
                 "Perturbarea echilibrului ecologic cauzată de defrișările necontrolate generează consecințe ireversibile asupra biodiversității și climatului global.",
                 "Diseminarea informațiilor eronate prin canale digitale poate conduce la dezinformare masivă și la erodarea încrederii publice în instituțiile fundamentale.",
                 "Complexitatea proceselor administrative influențează negativ eficiența deciziilor guvernamentale.",
                 "Polarizarea ideologică din societate poate compromite coeziunea comunitară și exacerba conflictele latente."]

for sentence in sentence_list:
    print(sentence_simplification(sentence))

Creșterea constantă în fața responsabilităților academice poate duce la o creștere a depresiei și la pierderea performanțelor intelectuale pe termen lung.
Implementarea unor strategii didactice inovatoare poate facilita utilizarea cunoștințelor teoretice într-un mod eficient și sustenabil.
Pierderea echilibrului ecologic cauzată de defriș ilegale are consecințe grave asupra biodiversității și mediului global.
Promovarea informațiilor eronate prin canale digitale poate conduce la manipulare masivă și la pierderea încrederii publice în instituțiile fundamentale.
Calitatea proceselor administrative influențează negativ eficiența deciziilor guvernamentale.
Presiunea ideologică din societate poate compromite coeziunea comunitară și agrava conflictele existente.


In [262]:
#special sentences
sentence_list = ["Au trecut peste 147 de ani de la înființarea Universității din București, iar eu am studiat acolo trei ani.", 
                 "Găsiți mai multe informații pe https://spacy.io/api/token.",]

for sentence in sentence_list:
    print(sentence_simplification(sentence))

Au trecut peste 147 de ani de la înființarea Universității din București, iar eu am studiat acolo trei ani.
Găsiți mai multe informații pe https://spacy.io/api/token.


In [261]:
text = "În contextul actual al dezvoltării societății contemporane, se observă o multitudine de problematici emergente care influențează semnificativ calitatea vieții cotidiene. Diversitatea factorilor determinanți contribuie la creșterea complexității soluțiilor propuse pentru ameliorarea acestor deficiențe. De exemplu, în domeniul educației, implementarea unor strategii pedagogice elaborate și utilizarea unor metodologii avansate au devenit priorități esențiale. Totuși, o parte considerabilă a populației rămâne marginalizată din cauza lipsei accesibilității resurselor educaționale. Pe de altă parte, în sectorul economic, dinamica pieței și globalizarea contribuie la apariția unor provocări substanțiale pentru întreprinderile mici și mijlocii. În acest sens, este imperativ să fie identificate modalități eficiente pentru stimularea sustenabilității acestor organizații. Un alt aspect relevant este cel legat de sănătate. Progresul tehnologic a facilitat crearea unor dispozitive medicale inovative, însă accesul la acestea este, din păcate, limitat pentru o parte semnificativă a populației."

print(text_simplification(text))

În contextul actual al dezvoltării societății contemporane, se observă o multitudine de probleme emergente care influențează semnificativ calitatea vieții cotidiene. Integrarea factorilor relevanți contribuie la creșterea calității soluțiilor propuse pentru ameliorarea acestor deficiențe. De exemplu, în domeniul educației, implementarea unor strategii didactice elaborat și utilizarea unor metode avansate au devenit priorități esențiale. Totuși, o parte considerabilă a populației rămâne pierdută din cauza lipsei disponibilității resurselor educaționale. Pe de altă parte, în sectorul economic, dinamica pieței și globalizarea contribuie la apariția unor provocări importante pentru întreprinderile mici și mijlocii. În acest sens, este esențial să fie identificat modalități eficiente pentru creșterea competitivității acestor organizații. Un alt aspect important este cel legat de sănătate. Progresu tehnologic a permis crearea unor dispozitive medicale noi, însă accesul la acestea este, din p